In [2]:
import pandas as pd
import numpy as np
interaction_path = "interactions.tsv"
warhead_path = "warhead.csv"

ligand_path = "ligands.csv"
chembl_path = "chembl.txt"
drug_bank_full_path = "drugbank_full.csv"
drug_bank_path = "drugbank.txt"
ncit_to_chebi_path = "NCIt-ChEBI_Mapping.txt"
rxcui_path = "rxcui.csv"

In [3]:
warhead_df = pd.read_csv(warhead_path, sep = ',', header = 0)
warhead_df

,Compound ID,Uniprot,Target,Name,Smiles,IC50 (nM),Assay (IC50),EC50 (nM),Assay (EC50),Kd (nM),...,Hydrogen Bond Acceptor Count,Hydrogen Bond Donor Count,Rotatable Bond Count,Topological Polar Surface Area,Molecular Formula,InChI,InChI Key,BindingDB,PubChem,ChEMBL
0,1,Q9NPI1,BRD7,BI-7273,COC1=CC(C2=CN(C)C(=O)C3=CN=CC=C23)=CC(OC)=C1CN...,117,BRD7 H3 AlphaScreen assay,NaN,NaN,0.3,...,6,0,5,56.59,C20H23N3O3,InChI=1S/C20H23N3O3/c1-22(2)11-17-18(25-4)8-13...,RBUYFHLQNPJMQM-UHFFFAOYSA-N,50183448.0,118796358.0,CHEMBL3823478
1,1,Q9H8M2,BRD9,BI-7273,COC1=CC(C2=CN(C)C(=O)C3=CN=CC=C23)=CC(OC)=C1CN...,19,BRD9-H3 AlphaScreen assay,NaN,NaN,15.4,...,6,0,5,56.59,C20H23N3O3,InChI=1S/C20H23N3O3/c1-22(2)11-17-18(25-4)8-13...,RBUYFHLQNPJMQM-UHFFFAOYSA-N,50183448.0,118796358.0,CHEMBL3823478
2,2,Q9NPI1,BRD7,BI-9564,COC1=CC(C2=CN(C)C(=O)C3=CN=CC=C23)=C(OC)C=C1CN...,3410,BRD7 H3 AlphaScreen assay,NaN,NaN,239/73,...,6,0,5,56.59,C20H23N3O3,InChI=1S/C20H23N3O3/c1-22(2)11-13-8-19(26-5)15...,BJFSUDWKXGMUKA-UHFFFAOYSA-N,50183449.0,117072549.0,CHEMBL3823101
3,2,Q9H8M2,BRD9,BI-9564,COC1=CC(C2=CN(C)C(=O)C3=CN=CC=C23)=C(OC)C=C1CN...,75,BRD9-H3 AlphaScreen assay,NaN,NaN,14.1,...,6,0,5,56.59,C20H23N3O3,InChI=1S/C20H23N3O3/c1-22(2)11-13-8-19(26-5)15...,BJFSUDWKXGMUKA-UHFFFAOYSA-N,50183449.0,117072549.0,CHEMBL3823101
4,3,Q07820,MCL1,A-1210477,CC1=NN(C)C(COC2=CC=C(N3CCN(S(=O)(=O)N(C)C)CC3)...,NaN,NaN,NaN,NaN,19,...,11,1,16,134.84,C46H55N7O7S,InChI=1S/C46H55N7O7S/c1-33-43(41(49(4)47-33)32...,XMVAWGSQPHFXKU-UHFFFAOYSA-N,50078163.0,66575373.0,CHEMBL3417704
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,378,P10275,AR W742C,NaN,C[C@H]1CC2(CCN(C3=CC=C(C(=O)N4CCN(C)CC4)C=C3)C...,NaN,NaN,NaN,NaN,NaN,...,5,0,3,53.82,C28H34ClN5O,InChI=1S/C28H34ClN5O/c1-21-18-28(20-34(21)25-8...,NWCHUXMIXFSRLT-NRFANRHFSA-N,NaN,NaN,NaN
1456,378,P10275,AR Del 873-879,NaN,C[C@H]1CC2(CCN(C3=CC=C(C(=O)N4CCN(C)CC4)C=C3)C...,NaN,NaN,NaN,NaN,NaN,...,5,0,3,53.82,C28H34ClN5O,InChI=1S/C28H34ClN5O/c1-21-18-28(20-34(21)25-8...,NWCHUXMIXFSRLT-NRFANRHFSA-N,NaN,NaN,NaN
1457,378,P10275,AR H875Y,NaN,C[C@H]1CC2(CCN(C3=CC=C(C(=O)N4CCN(C)CC4)C=C3)C...,NaN,NaN,NaN,NaN,NaN,...,5,0,3,53.82,C28H34ClN5O,InChI=1S/C28H34ClN5O/c1-21-18-28(20-34(21)25-8...,NWCHUXMIXFSRLT-NRFANRHFSA-N,NaN,NaN,NaN
1458,378,P10275,AR F877L,NaN,C[C@H]1CC2(CCN(C3=CC=C(C(=O)N4CCN(C)CC4)C=C3)C...,NaN,NaN,NaN,NaN,NaN,...,5,0,3,53.82,C28H34ClN5O,InChI=1S/C28H34ClN5O/c1-21-18-28(20-34(21)25-8...,NWCHUXMIXFSRLT-NRFANRHFSA-N,NaN,NaN,NaN


In [4]:
ligand_df = pd.read_csv(ligand_path, sep = ',', header = 1)
ligand_df

,Ligand ID,Name,Species,Type,Approved,Withdrawn,Labelled,Radioactive,PubChem SID,PubChem CID,...,Ligand Subunit Ensembl IDs,IUPAC name,INN,Synonyms,SMILES,InChIKey,InChI,GtoImmuPdb,GtoMPdb,Antibacterial
0,6069,"10,10-difluoro TXA<sub>2</sub>",NaN,Synthetic organic,NaN,NaN,NaN,NaN,178102690.0,73755171.0,...,NaN,"(Z)-7-[(1S,3R,4R,5R)-7,7-difluoro-3-[(E,3S)-3-...",NaN,NaN,CCCCC[C@@H](/C=C/[C@H]1C[C@@H]2O[C@H]([C@@H]1C...,CIYZUODQUHAGKZ-SCVXEUJASA-N,InChI=1S/C21H32F2O4/c1-2-3-6-9-16(24)13-12-15-...,NaN,NaN,NaN
1,13409,10-Me-Aplog-1,NaN,Synthetic organic,NaN,NaN,NaN,NaN,NaN,60155209.0,...,NaN,NaN,NaN,compound 1 [PMID: 32053979]|compound 5 [PMID: ...,[H][C@@]12CCC(C)(C)[C@]3(C[C@]([H])(OC(=O)C[C@...,SWKUTOXEFLDOID-PYCRRGESSA-N,InChI=1S/C28H40O8/c1-18-23(10-5-4-7-19-8-6-9-2...,NaN,NaN,NaN
2,1835,10-OBn-7&alpha;-F-gingkolide B,NaN,Natural product,NaN,NaN,NaN,NaN,135652650.0,73755010.0,...,NaN,NaN,NaN,10-OBn-7&alpha;-F-GB|compound 17 [PMID: 12570381],O=C1OC2C3([C@H]1OCc1ccccc1)C([C@@H](C1C43[C@@]...,LFVHCLPCEWRXME-CLNXZACASA-N,InChI=1S/C27H29FO10/c1-11-19(30)35-17-15(29)25...,NaN,NaN,NaN
3,1838,10-OBn-epi-ginkgolide C,NaN,Natural product,NaN,NaN,NaN,NaN,135652651.0,73755013.0,...,NaN,NaN,NaN,10-OBn-epi-GC,O=C1OC2C3([C@H]1OCc1ccccc1)C([C@@H](C1C43[C@@]...,YPOMAFCMSAKBRX-CLNXZACASA-N,InChI=1S/C27H30O11/c1-11-19(30)35-17-15(29)25-...,NaN,NaN,NaN
4,1836,10-OBn-ginkgolide B,NaN,Natural product,NaN,NaN,NaN,NaN,135652652.0,73755011.0,...,NaN,NaN,NaN,10-OBn-GB,O=C1OC2C3([C@H]1OCc1ccccc1)C(CC1C43[C@@](O2)(C...,PHJOWQWLQBFEEM-ZTPSGWAGSA-N,InChI=1S/C27H30O10/c1-12-19(29)35-17-16(28)25-...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12728,9627,ZY15557,NaN,Synthetic organic,NaN,NaN,NaN,NaN,340590257.0,90127874.0,...,NaN,"(2R,3S,5R)-2-(2,5-difluorophenyl)-5-(2,2-dioxo...",NaN,NaN,N[C@H]1C[C@H](CO[C@@H]1c1cc(F)ccc1F)N1CC2=C(C1...,ROFOCLOPXHFEIE-XOKHGSTOSA-N,InChI=1S/C17H20F2N2O3S/c18-12-1-2-15(19)14(3-1...,NaN,NaN,NaN
12729,11193,ZYBT1,NaN,Synthetic organic,NaN,NaN,NaN,NaN,434122253.0,118356682.0,...,NaN,"4-[4-amino-1-(2-prop-2-enoyl-3,3a,4,5,6,6a-hex...",NaN,example 13 [WO2015132799A2],C=CC(=O)N1CC2C(C1)CC(C2)n1nc(c2c1ncnc2N)c1ccc(...,YMCOZPHXLROTBV-UHFFFAOYSA-N,InChI=1S/C27H26N8O2/c1-2-22(36)34-13-18-11-20(...,NaN,NaN,NaN
12730,9193,zyklophin,NaN,Peptide,NaN,NaN,NaN,NaN,315661272.0,45483651.0,...,NaN,NaN,NaN,NaN,NCCCC[C@@H](C(=O)N)NC(=O)[C@@H]1CCCN1C(=O)[C@@...,UWLIKQFWZOPOAX-HRFULILYSA-N,InChI=1S/C65H96N20O13/c1-3-38(2)54-62(97)83-49...,NaN,NaN,NaN
12731,5302,Z-YVAD-FMK,NaN,Peptide,NaN,NaN,NaN,NaN,178101984.0,644196.0,...,NaN,methyl (3S)-5-fluoro-3-[[(2S)-2-[[(2S)-2-[[(2S...,NaN,benzyloxycarbonyl-Tyr-Val-Ala-Asp(OMe)-fluorom...,COC(=O)C[C@@H](C(=O)CF)NC(=O)[C@@H](NC(=O)[C@H...,MVPQJUFFTWWKBT-LBDWYMBGSA-N,InChI=1S/C31H39FN4O9/c1-18(2)27(30(42)33-19(3)...,NaN,NaN,NaN


In [5]:
ncit_to_chebi_df = pd.read_csv(ncit_to_chebi_path, sep = '\t', header = None)
ncit_to_chebi_df

,0,1
0,C1003,CHEBI:22565
1,C1005,CHEBI:30621
2,C1009,CHEBI:30969
3,C1011,CHEBI:3000
4,C1012,CHEBI:22720
...,...,...
3642,C99317,CHEBI:33343
3643,C99562,CHEBI:27827
3644,C997,CHEBI:27432
3645,C998,CHEBI:6912


In [6]:
drug_bank_df = pd.read_csv(drug_bank_path, sep = '\t', header = 0)
drug_bank_df

,source_id,Unnamed: 1
0,DB04459,1609.0
1,DB14882,24780387.0
2,DB17877,NaN
3,DB12318,10437018.0
4,DB00466,31304.0
...,...,...
3895,DB02552,445995.0
3896,DB02010,44259.0
3897,DB02010,44259.0
3898,DB16038,NaN


In [7]:
interaction_df = pd.read_csv(interaction_path, sep = '\t', header = 2)
interaction_df

,gene_claim_name,gene_concept_id,gene_name,drug_claim_name,drug_concept_id,drug_name,drug_is_approved,drug_is_immunotherapy,drug_is_antineoplastic,interaction_source_db_name,interaction_source_db_version,interaction_types,interaction_score,drug_specificity_score,gene_specificity_score,evidence_score
0,IGF2,hgnc:5466,IGF2,LINSITINIB,ncit:C70982,LINSITINIB,False,False,True,CIViC,26-November-2024,NaN,0.414316,0.518252,0.799449,1.0
1,IGF2,hgnc:5466,IGF2,DOCETAXEL,rxcui:1299922,DOCETAXEL ANHYDROUS,True,False,True,CIViC,26-November-2024,NaN,0.034942,0.043708,0.799449,1.0
2,IGF2,hgnc:5466,IGF2,CABAZITAXEL,rxcui:996051,CABAZITAXEL,True,False,True,CIViC,26-November-2024,NaN,0.131828,0.164898,0.799449,1.0
3,IGF2,hgnc:5466,IGF2,GEMCITABINE,rxcui:12574,GEMCITABINE,True,False,True,CIViC,26-November-2024,NaN,0.031524,0.039432,0.799449,1.0
4,IGF2,hgnc:5466,IGF2,GANITUMAB,ncit:C71531,GANITUMAB,False,True,True,CIViC,26-November-2024,NaN,0.483369,0.604627,0.799449,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98915,PTPRC,hgnc:9666,PTPRC,PROTEIN KINASE INHIBITOR,ncit:C1404,PROTEIN KINASE INHIBITOR,False,False,False,NCI,14-September-2017,NaN,1.160084,0.725553,0.799449,2.0
98916,PTPRC,hgnc:9666,PTPRC,OESTRADIOL,rxcui:24395,ESTRADIOL VALERATE,True,False,True,NCI,14-September-2017,NaN,0.123413,0.077186,0.799449,2.0
98917,PTPRC,hgnc:9666,PTPRC,PREDNISONE,rxcui:8640,PREDNISONE,True,False,True,NCI,14-September-2017,NaN,0.128898,0.080617,0.799449,2.0
98918,PTPRC,hgnc:9666,PTPRC,HEPARAN SULFATE,rxcui:2603494,HEPARAN SULFATE,False,False,True,NCI,14-September-2017,NaN,0.483369,0.302314,0.799449,2.0


In [8]:
interaction_df[['source', 'source_id']] = interaction_df['drug_concept_id'].str.split(':', expand=True)

In [9]:
interaction_df

,gene_claim_name,gene_concept_id,gene_name,drug_claim_name,drug_concept_id,drug_name,drug_is_approved,drug_is_immunotherapy,drug_is_antineoplastic,interaction_source_db_name,interaction_source_db_version,interaction_types,interaction_score,drug_specificity_score,gene_specificity_score,evidence_score,source,source_id
0,IGF2,hgnc:5466,IGF2,LINSITINIB,ncit:C70982,LINSITINIB,False,False,True,CIViC,26-November-2024,NaN,0.414316,0.518252,0.799449,1.0,ncit,C70982
1,IGF2,hgnc:5466,IGF2,DOCETAXEL,rxcui:1299922,DOCETAXEL ANHYDROUS,True,False,True,CIViC,26-November-2024,NaN,0.034942,0.043708,0.799449,1.0,rxcui,1299922
2,IGF2,hgnc:5466,IGF2,CABAZITAXEL,rxcui:996051,CABAZITAXEL,True,False,True,CIViC,26-November-2024,NaN,0.131828,0.164898,0.799449,1.0,rxcui,996051
3,IGF2,hgnc:5466,IGF2,GEMCITABINE,rxcui:12574,GEMCITABINE,True,False,True,CIViC,26-November-2024,NaN,0.031524,0.039432,0.799449,1.0,rxcui,12574
4,IGF2,hgnc:5466,IGF2,GANITUMAB,ncit:C71531,GANITUMAB,False,True,True,CIViC,26-November-2024,NaN,0.483369,0.604627,0.799449,1.0,ncit,C71531
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98915,PTPRC,hgnc:9666,PTPRC,PROTEIN KINASE INHIBITOR,ncit:C1404,PROTEIN KINASE INHIBITOR,False,False,False,NCI,14-September-2017,NaN,1.160084,0.725553,0.799449,2.0,ncit,C1404
98916,PTPRC,hgnc:9666,PTPRC,OESTRADIOL,rxcui:24395,ESTRADIOL VALERATE,True,False,True,NCI,14-September-2017,NaN,0.123413,0.077186,0.799449,2.0,rxcui,24395
98917,PTPRC,hgnc:9666,PTPRC,PREDNISONE,rxcui:8640,PREDNISONE,True,False,True,NCI,14-September-2017,NaN,0.128898,0.080617,0.799449,2.0,rxcui,8640
98918,PTPRC,hgnc:9666,PTPRC,HEPARAN SULFATE,rxcui:2603494,HEPARAN SULFATE,False,False,True,NCI,14-September-2017,NaN,0.483369,0.302314,0.799449,2.0,rxcui,2603494


In [10]:
info_before_counts = interaction_df['source'].value_counts()
info_before_counts

source
rxcui             39786
ncit              16279
iuphar.ligand     15085
chembl            13218
drugbank           3866
wikidata            117
hemonc               32
drugsatfda.nda       12
chemidplus            3
Name: count, dtype: int64

In [11]:
chembl_ind = interaction_df[interaction_df['source'] == 'chembl'].index
chembl_df = interaction_df.iloc[chembl_ind]
print('chembl: ',chembl_df.shape)

rxcui_ind = interaction_df[interaction_df['source'] == 'rxcui'].index
rxcui_df = interaction_df.iloc[rxcui_ind]
print('rxcui: ',rxcui_df.shape)

ncit_ind = interaction_df[interaction_df['source'] == 'ncit'].index
ncit_df = interaction_df.iloc[ncit_ind]
print('ncit: ',ncit_df.shape)

drugbank_ind = interaction_df[interaction_df['source'] == 'drugbank'].index
drugbank_df = interaction_df.iloc[drugbank_ind]
print('drugbank: ',drugbank_df.shape)

ligand_ind = interaction_df[interaction_df['source'] == 'iuphar.ligand'].index
ligand_df = interaction_df.iloc[ligand_ind]
print('ligand: ',ligand_df.shape)

wikidata_ind = interaction_df[interaction_df['source'] == 'wikidata'].index
wikidata_df = interaction_df.iloc[wikidata_ind]
print('wikidata: ',wikidata_df.shape)

chembl:  (13218, 18)
rxcui:  (39786, 18)
ncit:  (16279, 18)
drugbank:  (3866, 18)
ligand:  (15085, 18)
wikidata:  (117, 18)


In [12]:
#chembl mapping
chembl_df['source_id'].to_csv('chembl_df.csv', index=False)
wikidata_df['source_id'].to_csv('wikidata_df.csv', index=False)
drugbank_df['source_id'].to_csv('drugbank_df.csv', index=False)
ligand_df['source_id'].to_csv('ligand_df.csv', index=False)

In [13]:
# by using pubchem auto converter
chembl_map = pd.read_csv('chembl_map.txt', header = 0, sep = '\t')
drugbank_map = pd.read_csv('drugbank_map.txt', header = 0, sep = '\t')
ligand_map = pd.read_csv('ligand_map.txt', header = 0, sep = '\t')
wikidata_map = pd.read_csv('wikidata_map.txt', header = 0, sep = '\t')

chembl_map = chembl_map.set_index(chembl_ind)
interaction_df.loc[chembl_ind, 'Pubid'] = chembl_map['Unnamed: 1']

wikidata_map = wikidata_map.set_index(wikidata_ind)
interaction_df.loc[wikidata_ind, 'Pubid'] = wikidata_map['Unnamed: 1']

drugbank_map = drugbank_map.set_index(drugbank_ind)
interaction_df.loc[drugbank_ind, 'Pubid'] = drugbank_map['Unnamed: 1']

ligand_map = ligand_map.set_index(ligand_ind)
interaction_df.loc[ligand_ind, 'Pubid'] = ligand_map['Unnamed: 1']

interaction_df[interaction_df['source']=='iuphar.ligand']
interaction_df['Pubid'].notna().sum()

27428

In [14]:
#ncit
chebi_ind = interaction_df[interaction_df['source'] == 'ncit'].index
chebi_df = interaction_df.iloc[chebi_ind]

In [15]:
# Merge chebi_df with ncit_to_chebi_df
ncit_to_chebi_df= ncit_to_chebi_df.drop_duplicates(subset=ncit_to_chebi_df.columns[0])
ncit_chebi = pd.merge(chebi_df, ncit_to_chebi_df, left_on='source_id', right_on=ncit_to_chebi_df.columns[0], how='left')

# Split the values in column 1 and assign to new columns [2, 3]
ncit_chebi[['2', '3']] = ncit_chebi[1].str.split(':', expand=True)

# Fill NaN values in column 3 with -2
ncit_chebi['3'] = ncit_chebi['3'].fillna(-2)

# Read the mapping file
mapping = 'drug_links.csv'
db_map_df = pd.read_csv(mapping, sep=',', header=0)

# Extract relevant columns and fill NaN values with -1
chebi_map = db_map_df[['PubChem Compound ID', 'ChEBI ID']]
chebi_map = chebi_map.fillna(-1)

# Merge ncit_chebi with chebi_map based on column 3 and 'ChEBI ID'
chebi_pubid_re = pd.merge(ncit_chebi, chebi_map, left_on=ncit_chebi["3"].astype(int), right_on=chebi_map['ChEBI ID'].astype(int), how='left')
chebi_pubid_re = chebi_pubid_re.set_index(ncit_ind)

interaction_df.loc[chebi_ind, 'Pubid'] = chebi_pubid_re['PubChem Compound ID']
interaction_df['Pubid'].notna().sum()

28671

In [16]:
db_full_df = pd.read_csv(drug_bank_full_path, header = 0, sep = ',')
chebi_map2 = pd.merge(ncit_chebi, db_full_df, left_on = ncit_chebi["3"].astype(int), right_on = 'ChEBI ID', how = 'left')
chebi_map2 = chebi_map2.set_index(ncit_ind)

mask0 = interaction_df.loc[ncit_ind, 'Pubid'].isna()  # Mask for NaN values
interaction_df.loc[ncit_ind[mask0], 'Pubid'] = chebi_map2.loc[mask0, 'PubChem Compound ID']
(interaction_df['Pubid'].dropna() != -1).sum()

28340

In [17]:
# Pubchem
rxcui_df2 = pd.read_csv("rxcui_df.csv", sep = ',', header = None)
ncit_df2 = pd.read_csv("ncit_df.csv", sep = ',', header = None)

rxcui_map = pd.merge(rxcui_df, rxcui_df2, left_on=rxcui_df['source_id'].astype(int), right_on=rxcui_df2[0].astype(int), how='left')
ncit_map = pd.merge(ncit_df, ncit_df2, left_on='source_id', right_on=0, how='left')

ncit_map = ncit_map.set_index(ncit_ind)
rxcui_map = rxcui_map.set_index(rxcui_ind)

interaction_df.loc[rxcui_ind, 'Pubid'] = rxcui_map[1]
interaction_df.loc[ncit_ind, 'Pubid'] = ncit_map[1]
(interaction_df['Pubid'].dropna() != -1).sum()

67412

In [18]:
#Drugbank
db_full_df = pd.read_csv(drug_bank_full_path, header = 0, sep = ',')
drugbank_map2 = pd.merge(drugbank_df, db_full_df, left_on = 'source_id', right_on = 'DrugBank ID', how = 'left')
drugbank_map2 = drugbank_map2.set_index(drugbank_ind)
interaction_df.loc[drugbank_ind, 'Pubid'] = drugbank_map2['PubChem Compound ID']
(interaction_df['Pubid'].dropna() != -1).sum()

66915

In [19]:
# Github
mapping_full_df = pd.read_csv('drug-mappings.tsv', header=0, sep = '\t')
# pubchem_cid, drugbankId, chembl_id, chebi_id
mapping_full_df = mapping_full_df.drop_duplicates(subset = ['drugbankId'], keep='first')
interaction_df.loc[interaction_df['Pubid'] == -1, 'Pubid'] = None

db_map3 = pd.merge(drugbank_df, mapping_full_df, left_on = 'source_id', right_on = 'drugbankId', how = 'left')
db_map3 = db_map3.set_index(drugbank_ind)
# Update only rows with NaN in 'Pubid' for the given indices
mask = interaction_df.loc[drugbank_ind, 'Pubid'].isna()  # Mask for NaN values
interaction_df.loc[drugbank_ind[mask], 'Pubid'] = db_map3.loc[mask, 'pubchem_cid']

chembl_map2 = pd.merge(chembl_df, mapping_full_df, left_on = 'source_id', right_on = 'chembl_id', how = 'left')
chembl_map2 = chembl_map2.set_index(chembl_ind)
# Update only rows with NaN in 'Pubid' for the given indices
mask2 = interaction_df.loc[chembl_ind, 'Pubid'].isna()  # Mask for NaN values
interaction_df.loc[chembl_ind[mask2], 'Pubid'] = chembl_map2.loc[mask2, 'pubchem_cid']

mapping_full_df = mapping_full_df.drop_duplicates(subset = ['chebi_id'], keep='first')
chebi_map3 = pd.merge(ncit_chebi, mapping_full_df, left_on = ncit_chebi["3"], right_on = 'chebi_id', how = 'left')
chebi_map3 = chebi_map3.set_index(ncit_ind)
# Update only rows with NaN in 'Pubid' for the given indices
mask3 = interaction_df.loc[ncit_ind, 'Pubid'].isna()  # Mask for NaN values
interaction_df.loc[ncit_ind[mask3], 'Pubid'] = chebi_map3.loc[mask3, 'pubchem_cid']

(interaction_df['Pubid'].dropna() != -1).sum()

67565

In [20]:
wikidata_df['source_id'].to_csv('wiki.csv', index=False)

# Manual edit
interaction_df.loc[interaction_df['source_id'] == 'Q6816906,', 'Pubid'] = None

In [21]:
interaction_df.loc[interaction_df['source'] == 'chemidplus', "Pubid"]

6881    NaN
38481   NaN
86784   NaN
Name: Pubid, dtype: float64

In [25]:
merged_df = pd.merge(warhead_df, interaction_df, how='inner', left_on= warhead_df['PubChem'].fillna(-4), right_on=interaction_df['Pubid'].fillna(-3))
merged_df.drop_duplicates(subset = ['Pubid'], keep='first')

,key_0,Compound ID,Uniprot,Target,Name,Smiles,IC50 (nM),Assay (IC50),EC50 (nM),Assay (EC50),...,interaction_source_db_name,interaction_source_db_version,interaction_types,interaction_score,drug_specificity_score,gene_specificity_score,evidence_score,source,source_id,Pubid
0,118796358.0,1,Q9NPI1,BRD7,BI-7273,COC1=CC(C2=CN(C)C(=O)C3=CN=CC=C23)=CC(OC)=C1CN...,117,BRD7 H3 AlphaScreen assay,NaN,NaN,...,GuideToPharmacology,2024.3,inhibitor,6.525475,3.627763,1.798760,1.0,iuphar.ligand,9146,118796358.0
2,117072549.0,2,Q9NPI1,BRD7,BI-9564,COC1=CC(C2=CN(C)C(=O)C3=CN=CC=C23)=C(OC)C=C1CN...,3410,BRD7 H3 AlphaScreen assay,NaN,NaN,...,GuideToPharmacology,2024.3,inhibitor,8.700633,1.813882,4.796693,1.0,iuphar.ligand,8950,117072549.0
6,2733526.0,8,P03372,ER,Tamoxifen,CC/C(=C(\C1=CC=CC=C1)C1=CC=C(OCCN(C)C)C=C1)C1=...,61,Displacement of fluorescein-labeled estrogen f...,NaN,NaN,...,TEND,01-Aug-2011,NaN,0.074710,0.055812,0.167326,8.0,rxcui,10324,2733526.0
97,154257.0,9,P03372,ER,Bazedoxifene,CC1=C(C2=CC=C(O)C=C2)N(CC2=CC=C(OCCN3CCCCCC3)C...,23,Displacement of [3H]-17-beta-estradiol from ER...,NaN,NaN,...,TTD,2020.06.01,NaN,0.096674,0.725553,0.066621,2.0,rxcui,1441386,154257.0
104,216416.0,10,P03372,ER,Lasofoxifene,OC1=CC=C2C(=C1)CC[C@H](C1=CC=CC=C1)[C@@H]2C1=C...,4,Displacement of [3H]-17-beta-estradiol from ER...,NaN,NaN,...,TTD,2020.06.01,NaN,0.241684,1.209254,0.066621,3.0,ncit,C80679,216416.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27233,24795070.0,555,P11309,PIM1,SGI-1776,CN1CCC(CNC2=NN3C(C4=CC=CC(OC(F)(F)F)=C4)=CN=C3...,NaN,NaN,NaN,NaN,...,ChEMBL,34,inhibitor,0.497179,0.604627,0.411145,2.0,drugbank,DB12494,24795070.0
27247,58423153.0,556,P11309,PIM1,AZD1208,N[C@@H]1CCCN(C2=C(C3=CC=CC=C3)C=CC=C2/C=C2\SC(...,NaN,NaN,NaN,NaN,...,GuideToPharmacology,2024.3,inhibitor,0.170601,0.604627,0.282158,1.0,iuphar.ligand,7698,58423153.0
27253,44814409.0,557,P11309,PIM1,PIM-447,C[C@@H]1C[C@H](N)C[C@H](C2=CC=NC=C2NC(=O)C2=CC...,NaN,NaN,NaN,NaN,...,ChEMBL,34,inhibitor,0.497179,1.209254,0.411145,1.0,drugbank,DB14943,44814409.0
27263,5743.0,561,P04150,GR,Dexamethasone,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...,NaN,NaN,NaN,NaN,...,NCI,14-September-2017,NaN,0.064649,0.042680,0.757373,2.0,rxcui,3264,5743.0


In [23]:
merged_df.to_csv('result.csv')